### 훈민정음 서문을 학습하는 RNN
훈민정음 서문을 잘 학습했다면 서문의 첫 단어를 입력하면 다음 문장들을 자동으로 출력하는 Model구현이 Goal

In [4]:
import numpy as np 
import re # 보통 문자열을 다루기 위해서 import하는 모듈

In [7]:
data  = "나라의 말이 중국과 달라 문자와 서로  통하지 아니하기에 이런 까닭으로 어리석은 백성이 이르고자 할 바가 있어도 마침내 제 뜻을 히 펴지 못할 사람이 많으니라 내가 이를 위해 가엾이 여겨 새로 스물여덟 글자를 만드노니 사람마다 하여 쉬이 익혀 날로 쓰기에 편안케 하고자 할 따름이니라"

In [8]:
# data를 RNN이 쉽게 해석하기 위한 전처리 
def data_processting(data):
    data = re.sub('[^r가-힣]' , ' ' , data) # 한국어만 받아들이겠다 -> 한자나 , 영어등은 배제
    tokens = data.split() # 토큰을 이용해서 RNN이 torken by token으로 정의 
    vocab = list(set(tokens)) # NLTK가 tokenize 함수가 token화를 시키는 가장 유명한 func
    vocab_size = len(vocab) # 

    word_to_ix = {word : i for  i , word in enumerate(vocab)} # 모든 단어들을 0 ~ 99까지 독립적으로 배당하는 일 -> 숫자로 변경 후 one-hot incoding을 이용해서 벡터화 시킨 후 해당 벡터를 train data로 시용
    ix_to_word = {i : word for i , word in enumerate(vocab)} # model이 생성해낸 숫자를 자연어로 변경

    return tokens , vocab_size , word_to_ix , ix_to_word

In [9]:
# RNN의 3가지의 가중치
def init_weights(h_size , vocab_size):
    U = np.random.randn(h_size , vocab_size) * 0.01
    W = np.random.randn(h_size , h_size) * 0.01
    V = np.random.randn(vocab_size , h_size) * 0.01
    return U , W , V

In [11]:
# feedforward function
def feedforward(input , targets , hprev):
    loss = 0 
    xs , hs , ps , ys = {} ,{} ,{} ,{}
    hs[-1] : np.copy(hprev)
    for i in range(seq_len):
        xs[i] = np.zeros((vocab_size  , 1))
        xs[i][inputs[i]] = 1 # 각각의 word에 대한 one-hot coding
        hs[i] = np.tanh(np.dot(U , xs[i]) + np.dot(W , hs[i - 1])) 
        ys[i] = np.dot(V , hs[i])
        ps[i] = np.exp(ys[i] / np.sum(np.exp(ys[i]))) # softmax 계산
        loss += -np.log(ps[i][targets[i] , 0])
    return loss ,  ps , hs , xs

In [ ]:
def backward(ps , hs , xs):

    # Backward propagation through time(BPTT)
    # 처음에 모든 가중치들을 0으로 설정
    dV = np.zeros(V.shape)
    dW = np.zeros(W.shape)
    dU = np.zeros(U.shape)

    for i in range(seq_len)[::-1]:
        output = np.zeros((vocab_size , 1))
        output[targets[i]] = 1
        ps[i] = ps[i] - output.reshape(-1 , 1)
        # 매번 i스텝에서 dL / dVi를 구하기
        dV_step_i = ps[i] @ (hs[i]).T # (y_hat - y) @ hs.T - for each step

        dV = dV + dV_step_i # dL / dVi를 다 더하기 

        # 각i 별로 v와 w를 구하기 위해서는 
        # 먼저 공통적으로 계산되는 부분을 delta로 해서 계산해두고 
        # 그리고 시간을 거슬러 dL/dWij와 dL/dUij를 구한 뒤
        # 각각을 합하여 dL/dW 와 dL / dU를 구하고 
        # 다시 공통적으로 계산되는 delta를 업데이트 

        # i번째 스텝에서 공통적으로 사용될 delta
        delta_recent = (V.T @ ps[i]) * (1 - hs[i] ** 2)

        # 시간을 거슬러 올라가서 dL/dW 와 dL/dU를 구함
        for j in range(i + 1)[::-1]:
            dW_ij = delta_recent @ hs[j - 1].T

            dW = dW + dW_ij


            dU_ij = delta_recent @ xs[j].reshape(1,  -1)
            dU = dU + dU_ij

            # 그리고 다음
        